In [72]:
#find interactors
import pandas as pd
import numpy as np
import os
import csv
import json

In [101]:
yeastinhuman=pd.read_csv('yeastinhuman.csv')
humaninyeast=pd.read_csv('humaninyeast.csv')
Human_interactome = pd.read_csv('interactome_bi.csv')
HRR25_interactome=pd.read_csv('HRR25_interactome_all.csv')
yeastinhuman.head()


,Input Order,Search Term,Yeast GeneID,SGDID,Yeast Symbol,Species 2,Human GeneID,Human Species Gene ID,Human Symbol,Ensmbl ID (link HPA),DIOPT Score,Weighted Score,Rank,Best Score,Best Score Reverse,Prediction Derived From,Alignment & Scores,Feedback,Gene Details
0,1,RIO2,855514,S000005151,RIO2,Human,55781,18999,RIOK2,ENSG00000058729,13,12.73,high,Yes,Yes,"Compara, eggNOG, Hieranoid, Homologene, Inpara...",NaN,Add,G2F details (RIOK2) DRscDB Data: (RIOK2) ...
1,2,ENP1,852549,S000000451,ENP1,Human,705,1157,BYSL,ENSG00000112578,13,12.79,high,Yes,Yes,"Compara, eggNOG, Hieranoid, Homologene, Inpara...",NaN,Add,G2F details (BYSL) DRscDB Data: (BYSL) ...
2,3,DHH1,851394,S000002319,DHH1,Human,1656,2747,DDX6,ENSG00000110367,13,12.81,high,Yes,Yes,"eggNOG, Hieranoid, Homologene, Inparanoid, Iso...",NaN,Add,G2F details (DDX6) DRscDB Data: (DDX6) ...
3,4,PUF3,850647,S000003936,PUF3,Human,23369,14958,PUM2,ENSG00000055917,9,8.77,high,Yes,Yes,"Compara, eggNOG, Isobase, OrthoFinder, OrthoIn...",NaN,Add,G2F details (PUM2) DRscDB Data: (PUM2) ...
4,4,PUF3,850647,S000003936,PUF3,Human,9698,14957,PUM1,ENSG00000134644,8,7.82,moderate,No,Yes,"Compara, eggNOG, OrthoFinder, OrthoInspector, ...",NaN,Add,G2F details (PUM1) DRscDB Data: (PUM1) ...


In [74]:

def findortho1(interactome):
    human_database = {}
    for column in interactome.columns:
        if column != 'Interactors' and column != 'HRR25':
            human_database[column] = {}
            for interactor in range(len(interactome[column])):
                if interactome[column][interactor] >= 1.0:
                    human_database[column][interactome['Interactors'][interactor]] = (list(humaninyeast[humaninyeast['Human GeneID'] == int(interactome['Interactors'][interactor])]['Yeast Symbol'].values))
        elif column == 'HRR25':
            human_database[column] = {}
            for interactor in range(len(interactome[column])):
                if interactome[column][interactor] >= 1.0:
                    human_database[column][interactome['Interactors'][interactor]] = (list(yeastinhuman[yeastinhuman['Yeast Symbol'] == interactome['Interactors'][interactor]]['Human Symbol'].values))
                    
    df1 = pd.DataFrame.from_dict(human_database)
    return df1

In [75]:
temp=list(pd.array(humaninyeast['Human Symbol'][humaninyeast['Human GeneID']==5501]))
print(temp)

['PPP1CC']


In [114]:
Human_interactome
HRR25_interactome

,Interactors,HRR25
0,ABP1,1
1,ACO1,1
2,ADO1,1
3,AIM41,1
4,ALD6,1
...,...,...
253,YMR196W,1
254,YMR295C,1
255,YNL058C,1
256,YPT1,1


In [100]:
interactome = Human_interactome
cols=['Human_GeneID', 'Human_Gene_Symbol', 'CSNK1E', 'CSNK1A1','CSNK1D', 'CSNK1A1L', 'Yeast_Ortholog', 'Yeast_Ortholog_ID', 'HRR25_Interact']
human_database = pd.DataFrame(columns=cols)
entrez_id = pd.read_csv('entrez_id.csv')
for column in interactome.columns:
    if 'HRR25' not in interactome.columns:
        if column !='Interactors':
            for interactor in interactome['Interactors']:
                GeneID=interactor
                Gene_Symbol = list(entrez_id['Symbol'][entrez_id['ID']==interactor])
                if len(Gene_Symbol)==1:
                    Gene_Symbol = Gene_Symbol[0]
                csnk1e = int(Human_interactome['CSNK1E'][Human_interactome['Interactors']==interactor])
                csnk1a1 = int(Human_interactome['CSNK1A1'][Human_interactome['Interactors']==interactor])
                csnk1d = int(Human_interactome['CSNK1D'][Human_interactome['Interactors']==interactor])
                csnk1a1l = int(Human_interactome['CSNK1A1L'][Human_interactome['Interactors']==interactor])
                yeast_ortho = 'None'
                yeast_ortho_id = 'None'
                hrr25_interact = 'No'                
                if int(interactor) in humaninyeast['Human GeneID'].to_list():
                    yeast_ortho = list(humaninyeast['Yeast Symbol'][humaninyeast['Human GeneID']==int(interactor)])
                    yeast_ortho_id = list(humaninyeast['Yeast GeneID'][humaninyeast['Human GeneID']==int(interactor)])
                    for i in range(len(yeast_ortho)):
                        if yeast_ortho[i] in HRR25_interactome['Interactors'].to_list():
                            hrr25_interact = 'Yes'
                        temp = pd.DataFrame([[GeneID, Gene_Symbol, csnk1e, csnk1a1, csnk1d, csnk1a1l, yeast_ortho[i], yeast_ortho_id[i], hrr25_interact]], columns=cols)
                        human_database = pd.concat([human_database, temp])
                else:
                    temp = pd.DataFrame([[GeneID, Gene_Symbol, csnk1e, csnk1a1, csnk1d, csnk1a1l, yeast_ortho, yeast_ortho_id, hrr25_interact]], columns=cols)
                    human_database = pd.concat([human_database, temp])

                
human_database.to_csv('human_database.csv')
human_database

,GeneID,Gene_Symbol,CSNK1E,CSNK1A1,CSNK1D,CSNK1A1L,Yeast_Ortholog,Yeast_Ortholog_ID,HRR25_Interact
0,10010,TANK,0,0,0,0,None,None,No
0,100506658,OCLN,3,1,0,0,None,None,No
0,10112,KIF20A,0,0,0,0,None,None,No
0,102724631,POTEB3,0,0,0,0,None,None,No
0,10410,IFITM3,0,0,0,0,None,None,No
...,...,...,...,...,...,...,...,...,...
0,58494,JAM2,0,0,0,1,None,None,No
0,84687,PPP1R9B,0,0,0,0,None,None,No
0,150274,HSCB,0,0,0,0,None,None,No
0,409,ARRB2,0,0,0,0,None,None,No


In [113]:
interactome = HRR25_interactome
cols=['Yeast_GeneID', 'Yeast_Gene_Symbol','HRR25', 'Human_Ortholog', 'Human_Ortholog_ID', 'CSNK1E_Interact', 'CSNK1A1_Interact','CSNK1D_Interact', 'CSNK1A1L_Interact']
yeast_database = pd.DataFrame(columns=cols)
entrez_id = pd.read_csv('entrez_id.csv')
for column in interactome.columns:
    if column !='Interactors':
        for interactor in interactome['Interactors']:
            GeneID = 'None'
            entrez = list(entrez_id['ID'][entrez_id['Symbol']==interactor])
            Gene_Symbol = interactor
            if len(GeneID)==1:
                GeneID = entrez[0]
            hrr25 = int(HRR25_interactome['HRR25'][HRR25_interactome['Interactors']==interactor])
            human_ortho = 'None'
            human_ortho_id = '0'
            csnk1e_interact = 'No' 
            csnk1a1_interact = 'No'
            csnk1d_interact = 'No'
            csnk1a1l_interact = 'No'               
            if str(interactor) in yeastinhuman['Yeast Symbol'].to_list():
                human_ortho = list(yeastinhuman['Human Symbol'][yeastinhuman['Yeast Symbol']==str(interactor)])
                human_ortho_id = list(yeastinhuman['Human GeneID'][yeastinhuman['Yeast Symbol']==str(interactor)])
                for i in range(len(human_ortho)):
                    if human_ortho_id[i] in Human_interactome['Interactors'].to_list():
                        if (Human_interactome['CSNK1A1'][Human_interactome['Interactors']==int(human_ortho_id[i])] >= 1).any():
                            csnk1a1_interact = 'Yes'
                        if (Human_interactome['CSNK1A1L'][Human_interactome['Interactors']==int(human_ortho_id[i])] >= 1).any():
                            csnk1a1l_interact = 'Yes'
                        if (Human_interactome['CSNK1E'][Human_interactome['Interactors']==int(human_ortho_id[i])] >= 1).any():
                            csnk1e_interact = 'Yes'
                        if (Human_interactome['CSNK1D'][Human_interactome['Interactors']==int(human_ortho_id[i])] >= 1).any():
                            csnk1d_interact = 'Yes'
                    temp = pd.DataFrame([[GeneID, Gene_Symbol, hrr25, human_ortho[i], human_ortho_id[i], csnk1e_interact, csnk1a1_interact, csnk1d_interact, csnk1a1l_interact]], columns=cols)
                    yeast_database = pd.concat([yeast_database, temp])
            #else:
             #   temp = temp = pd.DataFrame([[GeneID, Gene_Symbol, hrr25, human_ortho[i], human_ortho_id[i], csnk1e_interact, csnk1a1_interact, csnk1d_interact, csnk1a1l_interact]], columns=cols)
              #  yeast_database = pd.concat([human_database, temp])

                
yeast_database.to_csv('yeast_database.csv')
yeast_database

,Yeast_GeneID,Yeast_Gene_Symbol,HRR25,Human_Ortholog,Human_Ortholog_ID,CSNK1E_Interact,CSNK1A1_Interact,CSNK1D_Interact,CSNK1A1L_Interact
0,None,ASC1,2,RACK1,10399,No,No,No,No
0,None,CCR4,1,CNOT6L,246175,No,No,No,No
0,None,CCR4,1,CNOT6,57472,No,No,No,No
0,None,CCR4,1,ANGEL1,23357,No,No,No,No
0,None,CRM1,1,XPO1,7514,No,No,No,No
...,...,...,...,...,...,...,...,...,...
0,None,UTP10,1,HEATR1,55127,No,No,No,No
0,None,VAM6,1,VPS39,23339,No,No,No,No
0,None,VAM6,1,TGFBRAP1,9392,No,No,No,No
0,None,YRA1,1,ALYREF,10189,No,No,No,No


In [23]:
HRR25_orthologues = findortho(HRR25_interactome)
Human_orthologues = findortho(Human_interactome)
HRR25_orthologues.to_csv('HRR25_orthologues.csv')
Human_orthologues.to_csv('Human_orthologues.csv')

HRR25_orthologues


,HRR25
ABP1,[]
ACO1,[]
ADO1,[]
AIM41,[]
ALD6,[]
...,...
YMR196W,[]
YMR295C,[]
YNL058C,[]
YPT1,[]


{'100506658': [],
 '1131': [],
 '114548': [],
 '23239': ['CYR1'],
 '2737': [],
 '2783': ['STE4'],
 '3183': ['NAB3'],
 '324': [],
 '3308': ['SSE1', 'SSE2'],
 '3320': ['HSP82', 'HSC82'],
 '5144': ['PDE2'],
 '54764': [],
 '5664': [],
 '5728': ['TEP1'],
 '7133': [],
 '7405': [],
 '8878': [],
 '94274': [],
 '993': ['MIH1', 'ARR2', 'YCH1'],
 '5501': ['GLC7'],
 '9958': ['UBP12']}